# Model Results - Comparison with v8.3.2


In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as ms
import sys

sys.path.insert(0, '..\..\Resources\Python-Functions')
import BigQuery
 
client = BigQuery.getBigQueryClient_TDMScenarios()

In [ ]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [ ]:
# FILTER
#strSQLWhere = ' WHERE NOT (t.scenarioID = 33)'
strSQLWhere = ''

# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share AS t" + strSQLWhere).to_dataframe()
#display(dfTransitSummary)

dfRouteSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS t" + strSQLWhere).to_dataframe()
#display(dfRouteSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios AS t" + strSQLWhere).to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge two dataframes
dfRouteSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfRouteSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData

In [ ]:
#Transit Ridership Plotting Function

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):

    data = []

    for v in tdmVersionsWithDate:
        for g in scenarioGroups:
            for m in modeGroups:

                # only do if data in dataframe since BY data is concatonated later
                if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                    # data for plotting from filtered dataframe}
                    plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin(lstIncludeInAll + [g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

                    #display(plotdata)

                    plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                    # fill any NaN values with zeros
                    plotdata = plotdata.fillna(0)

                    #display(plotdata)

                    xplot = plotdata['scenarioYear']
                    yplot = plotdata['TRIPS'       ]

                    trace1 = go.Scatter(
                        x=xplot,
                        y=yplot,
                        mode='markers+lines',
                        name= v + ' ' + g + ' ' + m,
                        marker=dict(size=12,
                                line=dict(width=2,
                                            color='DarkSlateGrey'))#,
                        #stackgroup='one',
                        #groupnorm='percent' # sets the normalization for the sum of the stackgroup
        #                line=dict(
        #                    shape='spline'
        #                )
                    )
                    data.append(trace1)


    layout = go.Layout(
        #title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips',
            rangemode = 'tozero'#,
            #range=(0,np.null)
        ),
        xaxis=dict(
            title='Year',
            range=(2018,2051)
        ),
        width=800,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    py.offline.iplot(fig)
py.offline.init_notebook_mode(connected=True)

In [ ]:
#Transit Share Plotting Function

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods):

    data = []

    modeGroups = sorted(modeGroups)

    for m in modeGroups: 
        # only do if data in dataframe since BY data is concatonated later
        if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) &
                                    (dfTransitSummaryPlotData['scenarioGroup'     ]==scenarioGroup     ) &
                                    (dfTransitSummaryPlotData['modeGroup'         ]==m                 ) &
                                    (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)    ) &
                                    (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)      )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                (dfTransitSummaryPlotData['modeGroup'         ]==m                                     ) &
                                                (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)                        ) &
                                                (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)                          )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['TRIPS'       ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= m,
                stackgroup='one'#,
                #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        #title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=950,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)
py.offline.init_notebook_mode(connected=True)

In [ ]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea_boardings(tdmVersionWithDate, scenarioGroup):

    data = []

    modes = sorted(dfRouteSummaryWithScenarioData['MODE'].unique())

    for m in modes: 
        # only do if data in dataframe since BY data is concatonated later
        if dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate) &
                                          (dfRouteSummaryWithScenarioData['scenarioGroup'     ]==scenarioGroup     ) &
                                          (dfRouteSummaryWithScenarioData['MODE'              ]==m                 )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                      (dfRouteSummaryWithScenarioData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                      (dfRouteSummaryWithScenarioData['MODE'              ]==m                                     )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(BOARDINGS=('DY_BRDA','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['BOARDINGS'   ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= 'Mode ' + str(m),
                stackgroup='one'#,
                #groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        #title=tdmVersionWithDate + ' Boardings by Mode',
        yaxis=dict(
            title='Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=840,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)
py.offline.init_notebook_mode(connected=True)

## Transit Comparisons

### Transit Ridership


In [ ]:
#| label: fig-hy-tr-all
#| fig-cap: Daily transit ridership - All Modes
#| cap-location: margin
#| column: screen-inset-right

update_plot(
    ['WF TDM v8.3.2 - 2022-02-04a','WF TDM v9.0 - 2022-12-19'], 
    ['RTP'], 
    ['4: Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

In [ ]:
#| label: fig-dy-tr-hbc
#| fig-cap: Daily transit ridership - HBC
#| cap-location: margin
#| column: screen-inset-right

update_plot(
    ['WF TDM v8.3.2 - 2022-02-04a','WF TDM v9.0 - 2022-12-19'], 
    ['RTP'], 
    ['4: Transit'], 
    ['HBC'], 
    ['Pk','Ok']
)

### Transit Share


In [ ]:
#| label: fig-shr-tr-all-832
#| fig-cap: Transit ridership share by transit mode - v8.3.2
#| cap-location: margin
#| column: screen-inset-right

update_plot_stackedarea(
    'WF TDM v8.3.2 - 2022-02-04a', 
    'RTP', 
    ['4a: Local Bus','4b: Core Bus','4c: Express Bus','4d: Bus-Rapid Transit','4e: Light-Rail Transit','4f: Commuter-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

In [ ]:
#| label: fig-shr-tr-all-9
#| fig-cap: Transit ridership share by transit mode - v9.0
#| cap-location: margin
#| column: screen-inset-right

update_plot_stackedarea(
    'WF TDM v9.0 - 2022-12-19', 
    'RTP', 
    ['4a: Local Bus','4b: Core Bus','4c: Express Bus','4d: Bus-Rapid Transit','4e: Light-Rail Transit','4f: Commuter-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

### Transit Boardings


In [ ]:
#| label: fig-brd-832
#| fig-cap: Transit boardings share by transit mode - v8.3.2
#| cap-location: margin
#| column: screen-inset-right

update_plot_stackedarea_boardings('WF TDM v8.3.2 - 2022-02-04a', 'RTP')

In [ ]:
#| label: fig-brd-9
#| fig-cap: Transit boardings share by transit mode - v9.0
#| cap-location: margin
#| column: screen-inset-right

update_plot_stackedarea_boardings('WF TDM v9.0 - 2022-12-19', 'RTP')

In [ ]:
#| label: fig-fr-brd
#| fig-cap: Daily FrontRunner boardings by station compared to observed.
#| cap-location: margin
#| column: screen-inset-right

boardings_barchart = pd.read_csv(r'D:\GitHub\TDM-Documentation\v9-whats-new\data\frontrunner-summary-comparison-v832-v9.csv')
import plotly.express as px
df = px.data.tips()
fig = px.histogram(boardings_barchart, x="Stop_Name", y="Boardings", text_auto='.2s',
             color='Model_Name', barmode='group',
             height=400)
fig.update_layout(
    xaxis_title="Station Name",
    yaxis_title="Average Boardings",
    legend_title="Model Version"
)
#fig.update_xaxes(tickangle=90)
fig.show()